# Assigment 1
### 02467 Computational Social Science Group 6

# Part 1: Web-scraping
### _Exercise: Web-scraping the list of participants to the International Conference in Computational Social Science_

In [ ]:
#code for part 1

### _How many unique researchers do you get?_
#### We got 1484 unique researchers for our answer.

### _Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words)._
#### We first cleaned the names by removing any special character and/or unnecessary white spaces and commas, and excluded names that were too short or contained numbers. We then removed the word "Chair" from the session chairs whose names were saved as "Chair: (name)". Lastly, we added these cleaned names into a list. Using the list, we retrieved the information needed using BeautifulSoup. We then sorted the list by the names to get our final answer.